In [1]:
import pandas as pd
from pathlib import Path
import re
import numpy as np
pd.set_option('display.max_columns', None)  # Alle Spalten anzeigen

## Erstellen der allgemeinen MeteoSuisse Tabelle

In [2]:
# ======================================================
# 1. Pfade zu den CSV-Dateien
# ======================================================
p_hist   = Path("ogd-smn_klo_t_historical_2020-2029.csv")
p_recent = Path("ogd-smn_klo_t_recent.csv")
p_params = Path("ogd-smn_meta_parameters.csv")

# ======================================================
# 2. Einstellungen
# ======================================================
ENCODING = "latin1"
SEP = ";"
TIME_COL = "reference_timestamp"
STATION_COL = "station_abbr"
TIME_START = pd.Timestamp("2023-01-01 05:50")
TIME_STOP  = pd.Timestamp("2025-09-12 23:20")

# ======================================================
# 3. Hilfsfunktion zur Spaltenvereinheitlichung
# ======================================================
def to_snake(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("/", " per ")
    s = s.replace("%", "percent")
    s = s.replace("°", "deg")
    s = re.sub(r"[^\w\[\]\(\)\-\s]", "", s)
    s = s.replace(" ", "_")
    s = re.sub(r"_+", "_", s).strip("_")
    return s

# ======================================================
# 4. Dateien laden
# ======================================================
hist   = pd.read_csv(p_hist,   sep=SEP, encoding=ENCODING)
recent = pd.read_csv(p_recent, sep=SEP, encoding=ENCODING)
params = pd.read_csv(p_params, sep=SEP, encoding=ENCODING)

# ======================================================
# 5. Zeitspalten parsen
# ======================================================
def normalize_parse_time(df, time_col=TIME_COL):
    df[time_col] = df[time_col].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
    df["time"] = pd.to_datetime(df[time_col], dayfirst=True, errors="coerce")
    return df

hist = normalize_parse_time(hist)
recent = normalize_parse_time(recent)

# ======================================================
# 6. Zusammenführen
# ======================================================
df_all = pd.concat([hist, recent], ignore_index=True)

# ======================================================
# 7. Lesbare Spaltennamen über Parameterdatei erzeugen
# ======================================================
params["parameter_description_en"] = params["parameter_description_en"].fillna(params["parameter_description_de"])
params["parameter_unit"] = params["parameter_unit"].fillna("")

def build_readable_name(row):
    desc = str(row["parameter_description_en"]).strip()
    unit = str(row["parameter_unit"]).strip()
    base = to_snake(desc)
    return f"{base}_[{to_snake(unit)}]" if unit else base

params["readable"] = params.apply(build_readable_name, axis=1)

# Mapping: Kurzname → lesbarer Name (nur für vorhandene Spalten)
present = set(df_all.columns)
short_to_readable = {
    r["parameter_shortname"]: r["readable"]
    for _, r in params.iterrows()
    if r["parameter_shortname"] in present
}

# Spalten umbenennen
rename_dict = {TIME_COL: "time_raw", STATION_COL: "station"}
rename_dict.update(short_to_readable)
df_all = df_all.rename(columns=rename_dict)

# ======================================================
# 8. Spaltenauswahl
# ======================================================
cols = ["station", "time"] + [c for c in df_all.columns if c not in ("station", "time", "time_raw")]
df_all = df_all[cols]

# ======================================================
# 9. Zeitraum filtern und sortieren
# ======================================================
mask = (df_all["time"] >= TIME_START) & (df_all["time"] <= TIME_STOP)
df_metar = (
    df_all.loc[mask]
    .sort_values(["time", "station"])
    .drop_duplicates(subset=["time", "station"], keep="last")
    .reset_index(drop=True)
)

# ======================================================
# 10. Ergebnis anzeigen (nicht speichern)
# ======================================================
print(f"✅ METAR-Tabelle erstellt — Zeilen: {len(df_metar):,}, Spalten: {len(df_metar.columns)}")
print(f"Zeitraum: {df_metar['time'].min()}  bis  {df_metar['time'].max()}\n")

# Tabelle ausgeben (z. B. in Jupyter oder VSCode direkt sichtbar)


✅ METAR-Tabelle erstellt — Zeilen: 141,946, Spalten: 31
Zeitraum: 2023-01-01 05:50:00  bis  2025-09-12 23:20:00



In [3]:
df_metar

,station,time,air_temperature_2_m_above_ground_current_value_[degc],air_temperature_at_5_cm_above_grass_current_value_[degc],air_temperature_at_surface_current_value_[degc],chill_temperature_current_value_[degc],relative_air_humidity_2_m_above_ground_current_value_[percent],dew_point_2_m_above_ground_current_value_[degc],vapour_pressure_2_m_above_ground_current_value_[hpa],atmospheric_pressure_at_barometric_altitude_(qfe)_current_value_[hpa],pressure_reduced_to_sea_level_according_to_standard_atmosphere_(qnh)_current_value_[hpa],pressure_reduced_to_sea_level_(qff)_current_value_[hpa],geopotential_height_of_the_850_hpa_level_current_value_[gpm],geopotential_height_of_the_700_hpa_level_current_value_[gpm],gust_peak_(one_second)_maximum_in_m_per_s_[m_per_s],wind_speed_vectoriel_ten_minutes_mean_in_m_per_s_[m_per_s],wind_speed_scalar_ten_minutes_mean_in_m_per_s_[m_per_s],wind_direction_ten_minutes_mean_[deg],foehn_index_[code],wind_speed_ten_minutes_mean_in_km_per_h_[km_per_h],gust_peak_(three_seconds)_maximum_in_m_per_s_[m_per_s],gust_peak_(one_second)_maximum_in_km_per_h_[km_per_h],gust_peak_(three_seconds)_maximum_in_km_per_h_[km_per_h],precipitation_ten_minutes_total_[mm],snow_depth_(automatic_measurement)_current_value_[cm],global_radiation_ten_minutes_mean_[w_per_m²],diffuse_radiation_ten_minutes_mean_[w_per_m²],longwave_incoming_radiation_ten_minutes_mean_[w_per_m²],longwave_outgoing_radiation_ten_minute_mean_[w_per_m²],shortwave_reflected_radiation_ten_minute_mean_[w_per_m²],sunshine_duration_ten_minutes_total_[min]
0,KLO,2023-01-01 05:50:00,13.0,9.9,9.1,13.0,66.3,6.9,9.9,973.0,1023.8,1023.5,NaN,NaN,7.3,4.0,4.1,224.0,NaN,14.8,6.9,26.3,24.8,0.0,0.0,2,1.0,302,NaN,NaN,0
1,KLO,2023-01-01 06:00:00,12.8,10.2,9.4,12.8,67.4,6.9,10.0,973.1,1023.9,1023.6,NaN,NaN,7.6,4.1,4.2,229.0,NaN,15.1,7.4,27.4,26.6,0.0,0.0,1,0.0,301,NaN,NaN,0
2,KLO,2023-01-01 06:10:00,12.1,10.1,9.4,12.1,70.5,6.9,9.9,973.1,1023.9,1023.8,NaN,NaN,6.6,4.4,4.4,234.0,NaN,15.8,6.2,23.8,22.3,0.0,0.0,2,1.0,301,NaN,NaN,0
3,KLO,2023-01-01 06:20:00,12.5,9.7,9.0,12.5,68.4,6.8,9.9,973.2,1024.0,1023.8,NaN,NaN,6.1,4.1,4.1,233.0,NaN,14.8,5.7,22.0,20.5,0.0,0.0,2,1.0,300,NaN,NaN,0
4,KLO,2023-01-01 06:30:00,12.6,9.7,9.0,12.6,67.6,6.8,9.9,973.1,1023.9,1023.7,NaN,NaN,6.7,4.3,4.3,221.0,NaN,15.5,6.5,24.1,23.4,0.0,0.0,2,1.0,300,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141941,KLO,2025-09-12 22:40:00,9.2,6.1,7.0,8.2,97.4,8.8,11.3,971.7,1022.4,1022.9,NaN,NaN,2.8,2.1,2.1,306.0,NaN,7.6,2.7,10.1,9.7,0.0,0.0,1,1.0,294,NaN,NaN,0
141942,KLO,2025-09-12 22:50:00,9.3,6.0,7.0,8.4,97.2,8.9,11.4,971.7,1022.4,1022.8,NaN,NaN,2.5,1.9,1.9,296.0,NaN,6.8,2.4,9.0,8.6,0.0,0.0,1,0.0,297,NaN,NaN,0
141943,KLO,2025-09-12 23:00:00,9.7,5.9,6.9,9.7,98.0,9.4,11.8,971.7,1022.4,1022.8,NaN,NaN,1.7,0.9,1.0,303.0,NaN,3.6,1.6,6.1,5.8,0.0,0.0,1,0.0,297,NaN,NaN,0
141944,KLO,2025-09-12 23:10:00,9.9,5.8,6.8,9.9,97.6,9.5,11.9,971.5,1022.2,1022.5,NaN,NaN,1.0,0.6,0.6,328.0,NaN,2.2,0.9,3.6,3.2,0.0,0.0,1,0.0,295,NaN,NaN,0


## Erstellen der Datei MeteoSuisse Daten_combined

In [4]:
# ======================================================
# 1. Pfade zu den Dateien anpassen
# ======================================================
p_hist   = Path("ogd-smn_klo_t_historical_2020-2029.csv")
p_recent = Path("ogd-smn_klo_t_recent.csv")
p_params = Path("ogd-smn_meta_parameters.csv")

# ======================================================
# 2. Einstellungen
# ======================================================
ENCODING = "latin1"
SEP = ";"
TIME_COL = "reference_timestamp"
STATION_COL = "station_abbr"

# Zeitraum für Filterung
TIME_START = pd.Timestamp("2023-01-01 05:50")
TIME_STOP  = pd.Timestamp("2025-09-12 23:20")

# ======================================================
# 3. Hilfsfunktion: Spaltennamen vereinheitlichen
# ======================================================
def to_snake(s: str) -> str:
    """Macht Text pythonfreundlich und lesbar."""
    s = s.strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("/", " per ")
    s = s.replace("%", "percent")
    s = s.replace("°", "deg")
    s = re.sub(r"[^\w\[\]\(\)\-\s]", "", s)
    s = s.replace(" ", "_")
    s = re.sub(r"_+", "_", s).strip("_")
    return s

# ======================================================
# 4. Daten laden
# ======================================================
hist   = pd.read_csv(p_hist,   sep=SEP, encoding=ENCODING)
recent = pd.read_csv(p_recent, sep=SEP, encoding=ENCODING)
params = pd.read_csv(p_params, sep=SEP, encoding=ENCODING)

# ======================================================
# 5. Zeitspalten parsen
# ======================================================
def normalize_parse_time(df, time_col=TIME_COL):
    df[time_col] = df[time_col].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
    # Unterstützt "dd.mm.yyyy HH:MM" und "dd.mm.yyyy HH:MM:SS"
    df["time"] = pd.to_datetime(df[time_col], dayfirst=True, errors="coerce")
    return df

hist = normalize_parse_time(hist)
recent = normalize_parse_time(recent)

# ======================================================
# 6. Tabellen kombinieren
# ======================================================
df_all = pd.concat([hist, recent], ignore_index=True)

# ======================================================
# 7. Parameterdatei: lesbare Spaltennamen generieren
# ======================================================
params["parameter_description_en"] = params["parameter_description_en"].fillna(params["parameter_description_de"])
params["parameter_unit"] = params["parameter_unit"].fillna("")

def build_readable_name(row):
    desc = str(row["parameter_description_en"]).strip()
    unit = str(row["parameter_unit"]).strip()
    base = to_snake(desc)
    return f"{base}_[{to_snake(unit)}]" if unit else base

params["readable"] = params.apply(build_readable_name, axis=1)

# Mapping Shortname → Beschreibung (nur vorhandene Spalten)
present = set(df_all.columns)
short_to_readable = {
    r["parameter_shortname"]: r["readable"]
    for _, r in params.iterrows()
    if r["parameter_shortname"] in present
}

# ======================================================
# 8. Spalten umbenennen
# ======================================================
rename_dict = {TIME_COL: "time_raw", STATION_COL: "station"}
rename_dict.update(short_to_readable)
df_all = df_all.rename(columns=rename_dict)

# ======================================================
# 9. Relevante Spalten auswählen
# ======================================================
cols = ["station", "time"] + [c for c in df_all.columns if c not in ("station", "time", "time_raw")]
df_all = df_all[cols]

# ======================================================
# 10. Zeitraumfilter & Sortierung (nur Halbstundentakt ab 05:50)
# ======================================================
# Grund-Filter Zeitraum
mask_range = (df_all["time"] >= TIME_START) & (df_all["time"] <= TIME_STOP)
df_metar = df_all.loc[mask_range].copy()

# Nur Zeitpunkte im :20 / :50 Raster behalten
mask_minute = df_metar["time"].dt.minute.isin([20, 50]) & (df_metar["time"].dt.second.fillna(0) == 0)
df_metar = df_metar.loc[mask_minute].copy()

# Sicherstellen, dass die Phase exakt zu 05:50 passt (alle 30 Minuten ab 05:50)
delta_sec = (df_metar["time"] - TIME_START).dt.total_seconds()
mask_phase = (delta_sec % (30 * 60) == 0)
df_metar = df_metar.loc[mask_phase].copy()

# Sortieren & Duplikate (falls mehrere Messungen je (time, station))
df_metar = (
    df_metar.sort_values(["time", "station"])
            .drop_duplicates(subset=["time", "station"], keep="last")
            .reset_index(drop=True)
)

# ======================================================
# 11. Ergebnis speichern
# ======================================================
output_file = Path("metar_combined_HALBHOUR_20230101_0550_to_20250912_2320.csv")
df_metar.to_csv(output_file, index=False, encoding="utf-8")

In [5]:
df_metar

,station,time,air_temperature_2_m_above_ground_current_value_[degc],air_temperature_at_5_cm_above_grass_current_value_[degc],air_temperature_at_surface_current_value_[degc],chill_temperature_current_value_[degc],relative_air_humidity_2_m_above_ground_current_value_[percent],dew_point_2_m_above_ground_current_value_[degc],vapour_pressure_2_m_above_ground_current_value_[hpa],atmospheric_pressure_at_barometric_altitude_(qfe)_current_value_[hpa],pressure_reduced_to_sea_level_according_to_standard_atmosphere_(qnh)_current_value_[hpa],pressure_reduced_to_sea_level_(qff)_current_value_[hpa],geopotential_height_of_the_850_hpa_level_current_value_[gpm],geopotential_height_of_the_700_hpa_level_current_value_[gpm],gust_peak_(one_second)_maximum_in_m_per_s_[m_per_s],wind_speed_vectoriel_ten_minutes_mean_in_m_per_s_[m_per_s],wind_speed_scalar_ten_minutes_mean_in_m_per_s_[m_per_s],wind_direction_ten_minutes_mean_[deg],foehn_index_[code],wind_speed_ten_minutes_mean_in_km_per_h_[km_per_h],gust_peak_(three_seconds)_maximum_in_m_per_s_[m_per_s],gust_peak_(one_second)_maximum_in_km_per_h_[km_per_h],gust_peak_(three_seconds)_maximum_in_km_per_h_[km_per_h],precipitation_ten_minutes_total_[mm],snow_depth_(automatic_measurement)_current_value_[cm],global_radiation_ten_minutes_mean_[w_per_m²],diffuse_radiation_ten_minutes_mean_[w_per_m²],longwave_incoming_radiation_ten_minutes_mean_[w_per_m²],longwave_outgoing_radiation_ten_minute_mean_[w_per_m²],shortwave_reflected_radiation_ten_minute_mean_[w_per_m²],sunshine_duration_ten_minutes_total_[min]
0,KLO,2023-01-01 05:50:00,13.0,9.9,9.1,13.0,66.3,6.9,9.9,973.0,1023.8,1023.5,NaN,NaN,7.3,4.0,4.1,224.0,NaN,14.8,6.9,26.3,24.8,0.0,0.0,2,1.0,302,NaN,NaN,0
1,KLO,2023-01-01 06:20:00,12.5,9.7,9.0,12.5,68.4,6.8,9.9,973.2,1024.0,1023.8,NaN,NaN,6.1,4.1,4.1,233.0,NaN,14.8,5.7,22.0,20.5,0.0,0.0,2,1.0,300,NaN,NaN,0
2,KLO,2023-01-01 06:50:00,13.1,10.8,10.1,13.1,65.4,6.8,9.9,973.2,1024.0,1023.7,NaN,NaN,9.3,6.2,6.3,231.0,NaN,22.7,9.1,33.5,32.8,0.0,0.0,1,0.0,300,NaN,NaN,0
3,KLO,2023-01-01 07:20:00,12.8,10.7,10.0,12.8,66.3,6.7,9.8,973.4,1024.2,1024.0,NaN,NaN,9.0,5.3,5.4,235.0,NaN,19.4,8.8,32.4,31.7,0.0,0.0,10,8.0,299,NaN,NaN,0
4,KLO,2023-01-01 07:50:00,13.4,12.3,11.7,13.4,64.0,6.7,9.8,973.7,1024.5,1024.2,NaN,NaN,12.7,7.2,7.3,228.0,NaN,26.3,12.2,45.7,43.9,0.0,0.0,40,25.0,300,NaN,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47311,KLO,2025-09-12 21:20:00,11.3,7.3,7.7,11.3,94.8,10.5,12.7,971.6,1022.3,1022.4,NaN,NaN,1.9,0.9,1.1,127.0,NaN,4.0,1.8,6.8,6.5,0.0,0.0,1,0.0,298,NaN,NaN,0
47312,KLO,2025-09-12 21:50:00,10.5,6.2,7.2,10.5,96.9,10.0,12.3,971.5,1022.2,1022.4,NaN,NaN,1.0,0.5,0.5,146.0,NaN,1.8,0.9,3.6,3.2,0.0,0.0,2,1.0,297,NaN,NaN,0
47313,KLO,2025-09-12 22:20:00,9.6,6.2,7.0,8.7,95.2,8.9,11.4,971.6,1022.3,1022.7,NaN,NaN,2.6,2.0,2.0,304.0,NaN,7.2,2.5,9.4,9.0,0.0,0.0,3,2.0,294,NaN,NaN,0
47314,KLO,2025-09-12 22:50:00,9.3,6.0,7.0,8.4,97.2,8.9,11.4,971.7,1022.4,1022.8,NaN,NaN,2.5,1.9,1.9,296.0,NaN,6.8,2.4,9.0,8.6,0.0,0.0,1,0.0,297,NaN,NaN,0
